# DTSC670: Foundations of Machine Learning Models
## Module 2
## Assignment 4: Custom Transformer and Transformation Pipeline

#### Name: Ejegu Smith

Begin by writing your name above.

Your task in this assignment is to create a custom transformation pipeline that takes in raw data and returns fully prepared, clean data that is ready for model training.  However, we will not actually train any models in this assignment.  This pipeline will employ an imputer class, a user-defined transformer class, and a data-normalization class.

Please note that the order of features in the final feature matrix must be correct.  See the below figure that illustrates the input and output of the transformation pipeline.  The positions of features $x_1$ and $x_2$ do not change - they remain in the first and second columns, respectvely, both before and after the transformation pipeline.  In the transformed dataset, the $x_5$ feature is next, and is followed by the newly computed feature $x_6$.  Finally, the last two columns are the remaining one-hot vectors obtained from encoding the categorical feature $x_3$.

<img src="DataTransformation.png " width ="500" />

# Import Data

Import data from the file called `CustomTransformerData.csv`.

In [1]:
import pandas as pd
fileName = 'CustomTransformerData.csv'
df = pd.read_csv(fileName)
df

,x1,x2,x3,x4,x5
0,1.5,2.354153,COLD,593,0.750000
1,2.5,3.314048,WARM,340,2.083333
2,3.5,4.021604,COLD,551,4.083333
3,4.5,NaN,COLD,2368,6.750000
4,5.5,5.847601,WARM,2636,10.083333
5,6.5,7.229910,WARM,2779,14.083333
6,7.5,7.997255,HOT,1057,18.750000
7,8.5,9.203947,COLD,819,24.083333
8,9.5,10.335348,WARM,3349,NaN
9,10.5,11.112142,HOT,3235,36.750000


# Create Custom Transformer

Create a custom transformer, just as we did in the lecture video entitled "Custom Transformers", that performs two computations: 

1. Adds an attribute to the end of the data (i.e. new last column) that is equal to $\frac{x_1^3}{x_5}$ for each observation

2. Drops the entire $x_4$ feature column.  (See further instructions below.)

You must name your custom transformer class `Assignment4Transformer`. Your class should include an input parameter with a default value of `True` that deletes the $x_4$ feature column when its value is `True`, but preserves the $x_4$ feature column when its value is `False`.

This transformer will be used in a pipeline. In that pipeline, an imputer will be run *before* this transformer. Keep in mind that the imputer will output an array, so **this transformer must be written to accept an array.**

Additionally, this transformer will ONLY be given the numerical features of the data. The categorical feature will be handled elsewhere in the full pipeline. This means that your code for this transformer **must reflect the absence of the categorical $x_3$ column** when indexing data structures.

In [78]:
from sklearn.base import BaseEstimator, TransformerMixin

# column index
x1, x2, x3, x4, x5  = 0, 1, 2, 3, 4


class Assignment4Transformer(BaseEstimator, TransformerMixin):
    def __init__(self, drop_x4=True, y=None):  # no *args or **kargs
        self.drop_x4 = drop_x4
        
    def fit(self, df, y=None):
        return self   # nothing else to do
    
    def transform(self, df):
        last_col = df[:, 0]**3 / df[:, 3]

        if self.drop_x4:
            df = np.delete(df,3, axis=1)
            
        return np.c_[df, last_col]

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x1      18 non-null     float64
 1   x2      16 non-null     float64
 2   x3      18 non-null     object 
 3   x4      18 non-null     int64  
 4   x5      17 non-null     float64
dtypes: float64(3), int64(1), object(1)
memory usage: 848.0+ bytes


In [ ]:
#dropping column x4
# creating dummy variables for column x3
#Assignment4Transformer = make_column_transformer(
           # ['drop', 'x4'],
           # ['ohe', 'x3'],
             # col_div, ['x3', 'x5'])

# Create Transformation Pipeline for Numerical Features

Create a custom transformation pipeline for numeric data only called `num_pipeline` that:

1. Applies the `SimpleImputer` class to the data, where the strategy is set to `mean`.

2. Applies the custom `Assignment4Transformer` class to the data.

3. Applies the `StandardScaler` class to the data.

In [80]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

imputer = SimpleImputer(strategy = 'median')
standard_scaler = StandardScaler()


In [81]:
num_pipeline = Pipeline([
                     ('imputer', imputer),  # first fill in the missing values w/ the median value
                      ('Assignment4Transformer', Assignment4Transformer()), # then define those combined attributes and add them to the data set 
                      ('std_scaler', standard_scaler)    # then standardize the data
])

# Create Numeric and Categorical DataFrames

Create two new data frames.  Create one DataFrame called `data_num` that holds the numeric features.  Create another DataFrame called `data_cat` that holds the categorical features.

In [82]:
data_cat = df[['x3']]
data_num = df.drop('x3', axis=1)

In [83]:
data_cat.head() # looking at data

,x3
0,COLD
1,WARM
2,COLD
3,COLD
4,WARM


In [84]:
data_num.head()  # looking at data

,x1,x2,x4,x5
0,1.5,2.354153,593,0.750000
1,2.5,3.314048,340,2.083333
2,3.5,4.021604,551,4.083333
3,4.5,NaN,2368,6.750000
4,5.5,5.847601,2636,10.083333


# Quick Testing

The full pipeline will be implemented with a `ColumnTransformer` class.  However, to be sure that our numeric pipeline is working properly, lets invoke the `fit_transform()` method of the `num_pipeline` object.  Then, take a look at the transformed data to be sure all is well.

### Run Pipeline and Create Transformed Numeric Data

In [91]:
from sklearn.compose import ColumnTransformer
import numpy as np

data_num_fit_transformed = num_pipeline.fit_transform(data_num)
data_num_fit_transformed

array([[-1.63835604, -1.73408102, -0.86175475, -1.61238454],
       [-1.44560827, -1.53051168, -1.10236978, -1.42053125],
       [-1.2528605 , -1.38045683, -0.90169875, -1.22867796],
       [-1.06011273,  0.04089724,  0.82635465, -1.03682467],
       [-0.86736496, -0.99320925,  1.08123539, -0.84497138],
       [-0.67461719, -0.70005654,  1.21723519, -0.6531181 ],
       [-0.48186942, -0.53732208, -0.42046869, -0.46126481],
       [-0.28912165, -0.28141344, -0.64681801, -0.26941152],
       [-0.09637388, -0.04147194,  1.7593323 , -0.40818975],
       [ 0.09637388,  0.12326643,  1.65091288,  0.11429506],
       [ 0.28912165,  0.26057817, -1.22029968,  0.30614835],
       [ 0.48186942,  0.44497389,  0.97947331,  0.49800164],
       [ 0.67461719,  0.75322293,  0.22434155,  0.68985493],
       [ 0.86736496,  0.87518476, -0.23311232,  0.88170822],
       [ 1.06011273,  0.04089724, -0.24167175,  1.07356151],
       [ 1.2528605 ,  1.41097779, -1.13090121,  1.2654148 ],
       [ 1.44560827,  1.

### One-Hot Encode Categorical Features

Similarly, you will employ a `OneHotEncoder` class in the `ColumnTransformer` below to construct the final full pipeline.  However, let's instantiate an object of the `OneHotEncoder` class called `cat_encoder` that has the `drop` parameter set to `first`.  Next, call the `fit_transform()` method and pass it your categorical data.  Take a look at the transformed one-hot vectors to be sure all is well.

In [92]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder(drop='first')

In [93]:
oneHot_cat = cat_encoder.fit_transform(data_cat) 

In [94]:
oneHot_cat.toarray() #checking

array([[0., 0.],
       [0., 1.],
       [0., 0.],
       [0., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.]])

In [95]:
cat_encoder.categories_  #checking

[array(['COLD', 'HOT', 'WARM'], dtype=object)]

# Put it All Together with a Column Transformer

Now, we are finally ready to construct the full transformation pipeline called `full_pipeline` that will transform our raw data into clean, ready-to-train data.  Construct this ColumnTransformer below, then call the `fit_transform()` method to obtain the final, clean data.  Save this output data into a variable called `data_trans`.

In [105]:
num_attribs = list(data_num)
cat_attribs = ["x3"]

In [106]:
full_pipeline = ColumnTransformer([("num", num_pipeline, num_attribs ), 
                                   ("cat", cat_encoder, cat_attribs )])           

In [107]:
data_trans = full_pipeline.fit_transform(df)
data_trans

/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


array([[-1.63835604, -1.73408102, -0.86175475, -1.61238454,  0.        ,
         0.        ],
       [-1.44560827, -1.53051168, -1.10236978, -1.42053125,  0.        ,
         1.        ],
       [-1.2528605 , -1.38045683, -0.90169875, -1.22867796,  0.        ,
         0.        ],
       [-1.06011273,  0.04089724,  0.82635465, -1.03682467,  0.        ,
         0.        ],
       [-0.86736496, -0.99320925,  1.08123539, -0.84497138,  0.        ,
         1.        ],
       [-0.67461719, -0.70005654,  1.21723519, -0.6531181 ,  0.        ,
         1.        ],
       [-0.48186942, -0.53732208, -0.42046869, -0.46126481,  1.        ,
         0.        ],
       [-0.28912165, -0.28141344, -0.64681801, -0.26941152,  0.        ,
         0.        ],
       [-0.09637388, -0.04147194,  1.7593323 , -0.40818975,  0.        ,
         1.        ],
       [ 0.09637388,  0.12326643,  1.65091288,  0.11429506,  1.        ,
         0.        ],
       [ 0.28912165,  0.26057817, -1.22029968,  0.

In [108]:
data_trans = pd.DataFrame(data_trans) # checking via dataframe
data_trans

,0,1,2,3,4,5
0,-1.638356,-1.734081,-0.861755,-1.612385,0.0,0.0
1,-1.445608,-1.530512,-1.102370,-1.420531,0.0,1.0
2,-1.252861,-1.380457,-0.901699,-1.228678,0.0,0.0
3,-1.060113,0.040897,0.826355,-1.036825,0.0,0.0
4,-0.867365,-0.993209,1.081235,-0.844971,0.0,1.0
5,-0.674617,-0.700057,1.217235,-0.653118,0.0,1.0
6,-0.481869,-0.537322,-0.420469,-0.461265,1.0,0.0
7,-0.289122,-0.281413,-0.646818,-0.269412,0.0,0.0
8,-0.096374,-0.041472,1.759332,-0.408190,0.0,1.0
9,0.096374,0.123266,1.650913,0.114295,1.0,0.0


# Prepare for Grading

Prepare your `data_trans` NumPy array for grading by using the NumPy [around()](https://numpy.org/doc/stable/reference/generated/numpy.around.html) function to round all the values to 2 decimal places - this will return a NumPy array.

Please note the final order of the features in your final numpy array, which is given at the top of this document.

___You MUST print your final answer, which is the NumPy array discussed above, using the `print()` function!  This MUST be the only `print()` statement in the entire notebook!  Do not print anything else using the print() function in this notebook!___

In [109]:
print(np.around(data_trans,decimals=2))

       0     1     2     3    4    5
0  -1.64 -1.73 -0.86 -1.61  0.0  0.0
1  -1.45 -1.53 -1.10 -1.42  0.0  1.0
2  -1.25 -1.38 -0.90 -1.23  0.0  0.0
3  -1.06  0.04  0.83 -1.04  0.0  0.0
4  -0.87 -0.99  1.08 -0.84  0.0  1.0
5  -0.67 -0.70  1.22 -0.65  0.0  1.0
6  -0.48 -0.54 -0.42 -0.46  1.0  0.0
7  -0.29 -0.28 -0.65 -0.27  0.0  0.0
8  -0.10 -0.04  1.76 -0.41  0.0  1.0
9   0.10  0.12  1.65  0.11  1.0  0.0
10  0.29  0.26 -1.22  0.31  0.0  1.0
11  0.48  0.44  0.98  0.50  0.0  1.0
12  0.67  0.75  0.22  0.69  0.0  0.0
13  0.87  0.88 -0.23  0.88  1.0  0.0
14  1.06  0.04 -0.24  1.07  1.0  0.0
15  1.25  1.41 -1.13  1.27  0.0  1.0
16  1.45  1.54 -1.23  1.46  1.0  0.0
17  1.64  1.71  0.25  1.65  1.0  0.0
